In [1]:
import pdfplumber

In [ ]:
# def extract_text_pdfplumber(pdf_path: Path, start_page: int, end_page: int) -> dict[int, str]:
#     """
#     Extract text from [start_page, end_page] inclusive.
#     Pages are 1-indexed for user convenience.

#     Returns:
#         {page_number: page_text}
#     """
#     if start_page < 1:
#         raise ValueError("start_page must be >= 1")
#     if end_page < start_page:
#         raise ValueError("end_page must be >= start_page")

#     parts: dict[int, str] = {}
#     with pdfplumber.open(str(pdf_path)) as pdf:
#         total_pages = len(pdf.pages)
#         if end_page > total_pages:
#             raise ValueError(f"end_page ({end_page}) exceeds total pages ({total_pages})")

#         for page_num in range(start_page, end_page + 1):
#             page = pdf.pages[page_num - 1]  # pdf.pages is 0-indexed
#             text = (page.extract_text() or "").strip()
#             parts[page_num] = text

#     return parts

In [2]:
# extract text from pdf 
pdf_path = "AI_Engineering_Building_Applications_Chip_Huyen.pdf"

parts = {}
with pdfplumber.open(str(pdf_path)) as pdf:
    total_pages = len(pdf.pages)
    
    for page_num in range(1, 7):
        page = pdf.pages[page_num-1]
        text = (page.extract_text() or "")
        parts[page_num] = {
            "chapter": 4, 
            "text": text
        }
parts

{1: {'chapter': 4,
  'text': 'Chapter 4. Evaluate AI Systems\nA model is only useful if it works for its intended purposes. You need to\nevaluate models in the context of your application. Chapter 3 discusses\ndifferent approaches to automatic evaluation. This chapter discusses how to\nuse these approaches to evaluate models for your applications.\nThis chapter contains three parts. It starts with a discussion of the criteria\nyou might use to evaluate your applications and how these criteria are\ndefined and calculated. For example, many people worry about AI making\nup facts—how is factual consistency detected? How are domain-specific\ncapabilities like math, science, reasoning, and summarization measured?\nThe second part focuses on model selection. Given an increasing number of\nfoundation models to choose from, it can feel overwhelming to choose the\nright model for your application. Thousands of benchmarks have been\nintroduced to evaluate these models along different criteria. C

- `sections_re` splits whenever the preceding character is . ! or ? and the break contains one or more \n (optional whitespace in between). Adjust the regex if you want other punctuation or need to keep blank lines.

In [ ]:
# # paragraph chunking
# import re
# from typing import Dict, List

# SECTION_BREAK_RE = re.compile(r"([.!?]\s*\n+)")


# class SectionChunker:
#     def __init__(self, parts: Dict[int, dict]):
#         self.parts = parts
#         self.state = {"buffer": [], "pages": [], "section_id": 0}
#         self.sections: List[dict] = []

#     @staticmethod
#     def flush(state: dict, parts: Dict[int, dict], sections: List[dict]) -> None:
#         text = "".join(state["buffer"]).strip()
#         pages = state["pages"]
#         if not text or not pages:
#             state["buffer"].clear()
#             state["pages"].clear()
#             return
#         state["section_id"] += 1
#         sections.append(
#             {   "chapter": parts[pages[0]]["chapter"],
#                 "pages": pages.copy(),
#                 "section": state["section_id"],
#                 "text": text,
#             }
#         )
#         state["buffer"].clear()
#         state["pages"].clear()

#     def chunk_sections(self) -> List[dict]:
#         for page_num in sorted(self.parts):
#             text = self.parts[page_num].get("text", "")
#             if not text:
#                 continue

#             for piece in SECTION_BREAK_RE.split(text):
#                 if not piece:
#                     continue
#                 if SECTION_BREAK_RE.fullmatch(piece):
#                     self.state["buffer"].append(piece.rstrip("\n"))
#                     SectionChunker.flush(self.state, self.parts, self.sections)
#                 else:
#                     if not self.state["pages"] or self.state["pages"][-1] != page_num:
#                         self.state["pages"].append(page_num)
#                     self.state["buffer"].append(piece)

#         SectionChunker.flush(self.state, self.parts, self.sections)
#         return self.sections

# chunker = SectionChunker(parts)
# sections = chunker.chunk_sections()
# sections


In [ ]:
# paragraph chunking - use this one 

import re 

SECTION_BREAK_RE = re.compile(r"([.!?]\s*\n+)")

sections = []
current_section_id = 0

for page_num, payload in parts.items():
    text = payload["text"]
    pieces = SECTION_BREAK_RE.split(text)

    buffer = ""
    for piece in pieces:
        if SECTION_BREAK_RE.fullmatch(piece):
            buffer += piece.strip("\n")  # keep punctuation/whitespace
            current_section_id += 1
            sections.append(
                {
                    "chapter": payload["chapter"],
                    "pages": [page_num],
                    "section": current_section_id,
                    "text": buffer.strip(),
                }
            )
            buffer = ""
        else:
            buffer += piece

    if buffer.strip():
        current_section_id += 1
        sections.append(
            {
                "section": current_section_id,
                "pages": [page_num],
                "chapter": payload["chapter"],
                "text": buffer.strip(),
            }
        )
sections

In [5]:
parts = {}
with pdfplumber.open(str(pdf_path)) as pdf:
    total_pages = len(pdf.pages)

    for page_num in range(1, 7):
        page = pdf.pages[page_num - 1]
        text = page.extract_text() or ""
        parts[page_num] = {
            "chapter": 4,
            "text": text,
        }

# collapse into a single entry
all_text = []
for page_num in sorted(parts):
    payload = parts[page_num]
    all_text.append(payload["text"])

combined = {
    "chapter": 4,         # or derive from whatever logic you need
    "text": "\n".join(all_text),
}

# print(combined["chapter"])
# print(len(combined["text"]))
chap4 = [combined]

In [8]:

def sliding_window(seq, size, step):
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    n = len(seq)
    result = []
    for i in range(0, n, step):
        chunk = seq[i:i+size]
        result.append({'start': i, 'text': chunk})
        if i + size >= n:
            break

    return result

def chunk_documents(docs:list, size=500, step=300) -> list:
    doc_chunks = []

    for doc in docs:
        doc_copy = doc.copy()
        doc_content = doc_copy.pop('text')
        chunks = sliding_window(doc_content, size, step)
        for chunk in chunks:
            chunk.update(doc_copy)
        doc_chunks.extend(chunks)
    return doc_chunks

chunks = chunk_documents(chap4)

In [9]:
from minsearch import VectorSearch
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm

embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')
v_index = VectorSearch(keyword_fields = [])

def create_doc_embeddings(chunks:list):
    embeddings = []

    for d in tqdm(chunks):
        v = embedding_model.encode(d['text'])
        embeddings.append(v)

    return np.array(embeddings)


def create_vector_index(chunks:list):
    emb_array = create_doc_embeddings(chunks)
    return v_index.fit(emb_array, chunks)



def text_embedding_search(query:str):
    query_embedding = embedding_model.encode(query)
    return v_index.search(query_embedding, num_results=5)


/Users/vancescadinh/Documents/AI/ai-engineering-notes/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
vector_store = create_vector_index(chunks)

100%|██████████| 25/25 [00:00<00:00, 25.89it/s]
